## Final Project Submission

Please fill out: 
* Student name: Lynette Wangari
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: Asha Deen
* Blog post URL:


# MICROSOFT MOVIE STUDIO EXPLORATION AND DATA ANALYSIS

### 1) Introduction

Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies.  

Our task is to analyze the current movie genres at the Box office and their performance, considering various factors that affect movie success.

This analysis will  provide insights to help Microsoft make informed decisions on which types of films to focus on. To start, we'll gather and analyze relevant data and clarify business objectives. This will establish a solid foundation and direction for the project.

### 2) Problem Statement
Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. 

You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of Microsoft's new movie studio can use to help decide what type of films to create.

### 3) Data Sets Used
The movie datasets are from the following websites:
1. https://www.boxofficemojo.com/
2. https://www.imdb.com/
3. https://www.rottentomatoes.com/
4. https://www.themoviedb.org/
5. https://www.the-numbers.com/


### 4) Project Structure

1. Data Understanding
2. Data Cleaning and Preparation
3. Exploratory Data Analysis
4. Visualization: Insights Derived from the Values
5. Recommendations: Key findings

Data Cleaning and Preparation
Exploratory Data Analysis
Visualization: Insights Derived from the Values
Recommendations: Key findings


#### 1. Data Understanding
 Our data is stored in a folder called zippedData. So we will first import the necessary packages then we'll explore the data by loading the datasets. 

In [71]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mticker
import seaborn as sns
%matplotlib inline

In [72]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('zippedData/im.db')

# Create a cursor object
cur = conn.cursor()

# Execute the SQL query to fetch the list of tables
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all table names
tables = cur.fetchall()

# Print the list of tables
print(tables)

# Close the connection
conn.close()

[('movie_basics',), ('directors',), ('known_for',), ('movie_akas',), ('movie_ratings',), ('persons',), ('principals',), ('writers',)]


In [73]:
#import data from db
bom_movie_df = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
reviews_df = pd.read_csv('./zippedData/rt.reviews.tsv.gz',sep="\t", encoding = 'unicode_escape')
tmdb_df = pd.read_csv('./zippedData/tmdb.movies.csv.gz')
budget_df = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')


In [74]:
import sqlite3
import pandas as pd

# Path to the SQLite database file
db_file_path = r'C:\Users\User\Documents\GitHub\dsc-phase-1-project-v2-4\zippedData\im.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_file_path)

# Read data from the movie_basics table
movie_basics = pd.read_sql("""
    SELECT *
    FROM movie_basics
""", conn)

# Display the first few rows of the DataFrame
movie_basics.head()



,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [75]:
movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [76]:
movie_ratings = pd.read_sql("""
    SELECT *
    FROM movie_ratings
""", conn)
movie_ratings.head(100)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
95,tt1515069,8.3,15
96,tt1515208,6.3,1447
97,tt1516577,5.3,147
98,tt1516595,4.4,64


In [77]:
movie_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


#### 2. Data Cleaning and Preparation


In [78]:
#cleaning data from Box Office Mojo
#fill missing values with zero
bom_movie_df['foreign_gross'] = bom_movie_df['foreign_gross'].fillna('0')

#remove commas from foreign gross and convert them to float
bom_movie_df['foreign_gross'] = bom_movie_df['foreign_gross'].str.replace(',', '').astype(float)

# Convert 'domestic_gross' column to float
bom_movie_df['domestic_gross'] = bom_movie_df['domestic_gross'].astype(float)

# Reorganize the columns
bom_movie_df = bom_movie_df[['title', 'year', 'domestic_gross', 'foreign_gross']]

In [79]:
bom_movie_df.head()

,title,year,domestic_gross,foreign_gross
0,Toy Story 3,2010,415000000.0,652000000.0
1,Alice in Wonderland (2010),2010,334200000.0,691300000.0
2,Harry Potter and the Deathly Hallows Part 1,2010,296000000.0,664300000.0
3,Inception,2010,292600000.0,535700000.0
4,Shrek Forever After,2010,238700000.0,513900000.0


In [80]:
# cleaning data from Movie Budgets
#  Convert 'release_date' column to datetime format
budget_df['release_date'] = pd.to_datetime(budget_df['release_date'])

# Convert 'production_budget', 'domestic_gross', and 'worldwide_gross' columns to integer format
def clean_currency(x):
   if isinstance(x, str):
        return int(x.replace('$','').replace(',',''))
   else:
        return x

budget_df['production_budget'] = budget_df['production_budget'].map(clean_currency)
budget_df['domestic_gross'] = budget_df['domestic_gross'].map(clean_currency)
budget_df['worldwide_gross'] = budget_df['worldwide_gross'].map(clean_currency)

# Reorganize columns for better readability
budget_df = budget_df[['movie', 'release_date', 'production_budget', 'domestic_gross', 'worldwide_gross']]

In [81]:
budget_df.head()

,movie,release_date,production_budget,domestic_gross,worldwide_gross
0,Avatar,2009-12-18,425000000,760507625,2776345279
1,Pirates of the Caribbean: On Stranger Tides,2011-05-20,410600000,241063875,1045663875
2,Dark Phoenix,2019-06-07,350000000,42762350,149762350
3,Avengers: Age of Ultron,2015-05-01,330600000,459005868,1403013963
4,Star Wars Ep. VIII: The Last Jedi,2017-12-15,317000000,620181382,1316721747


In [82]:
#cleaning movie basics
movie_basics.isna().sum()

#drop the rows
movie_basics['runtime_minutes'].dropna()
movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


Merge Data sets to get information on movie, title release date, runtime, votes, actor and director

In [83]:
#join the movie basics table and movie ratings
movie_basics_ratings_merge = pd.read_sql('''
    SELECT 
        movie_id,
        primary_title AS title,
        start_year AS year,
        runtime_minutes AS runtime,
        genres,
        averagerating AS avg_rating,
        numvotes
    FROM 
        movie_basics
    LEFT JOIN 
        movie_ratings
    USING(movie_id)
''', conn)
movie_basics_ratings_merge.head()

,movie_id,title,year,runtime,genres,avg_rating,numvotes
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43.0
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
3,tt0069204,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
4,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


In [84]:
#merge the principals and persons tables
directors = pd.read_sql('''
    SELECT 
        principals.movie_id, 
        persons.primary_name AS director
    FROM 
        principals
    JOIN 
        persons
    USING(person_id)
    WHERE 
        category = 'director'
''', conn)

directors.head()

,movie_id,director
0,tt0111414,Frank Howson
1,tt0323808,Robin Hardy
2,tt0417610,Alejandro Chomski
3,tt0469152,Alyssa R. Bennett
4,tt0473032,J. Neil Schulman


In [85]:
#group the directors
directors = directors.drop_duplicates()

#define a separator for joining 
separator = ','

#group the directors by movie id and concatenate their names 
directors = directors.groupby('movie_id').agg(lambda x: separator.join(x.tolist())).reset_index()

In [86]:
directors.head(20)

,movie_id,director
0,tt0063540,Harnam Singh Rawail
1,tt0066787,Mani Kaul
2,tt0069049,Orson Welles
3,tt0069204,Hrishikesh Mukherjee
4,tt0100275,"Raoul Ruiz,Valeria Sarmiento"
5,tt0111414,Frank Howson
6,tt0112502,Mc Jones
7,tt0139613,"António Loja Neves,José Manuel Alves Pereira"
8,tt0146592,Ágnes Kocsis
9,tt0154039,Miklós Jancsó


In [87]:
#get actors and from the principals and actors table
# Select actors from principals table where the category is 'actor' and order by movie_id
actors_query = '''
SELECT movie_id, primary_name AS actors
FROM principals
JOIN persons USING(person_id)
WHERE category = 'actor'
ORDER BY movie_id
'''

# Read data from SQL query into a DataFrame
actors = pd.read_sql(actors_query, conn)

# Remove duplicate entries
actors = actors.drop_duplicates()

# Group actors by movie_id and concatenate their names into a single string separated by ','
actors_grouped = actors.groupby('movie_id')['actors'].apply(lambda x: ', '.join(x)).reset_index()


In [88]:
actors.head(20)

,movie_id,actors
0,tt0063540,Dilip Kumar
1,tt0063540,Balraj Sahni
2,tt0063540,Sanjeev Kumar
3,tt0066787,Arun Khopkar
4,tt0066787,Om Shivpuri
5,tt0069049,John Huston
6,tt0069049,Peter Bogdanovich
7,tt0069204,Rabi Ghosh
8,tt0069204,Vijay Arora
9,tt0069204,Asrani


In [97]:
#merge the dataframe containing movie information with that containing directord
merged_df_directors = pd.merge(movie_basics_ratings_merge ,directors, how = 'left', on = 'movie_id')
    


In [100]:
merged_df_directors.head()

,movie_id,title,year,runtime,genres,avg_rating,numvotes,director
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Harnam Singh Rawail
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43.0,Mani Kaul
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,Orson Welles
3,tt0069204,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,Hrishikesh Mukherjee
4,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,"Raoul Ruiz,Valeria Sarmiento"


In [101]:
sql_df = pd.merge(merged_df_directors, actors, how = 'left', on = 'movie_id')

In [102]:
sql_df.head()

,movie_id,title,year,runtime,genres,avg_rating,numvotes,director,actors
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Harnam Singh Rawail,Dilip Kumar
1,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Harnam Singh Rawail,Balraj Sahni
2,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Harnam Singh Rawail,Sanjeev Kumar
3,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43.0,Mani Kaul,Arun Khopkar
4,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43.0,Mani Kaul,Om Shivpuri


In [107]:

sql_df.dropna(subset=['genres'], inplace = True)
#remove duplicates
sql_df.drop_duplicates(inplace=True)

In [108]:
sql_df.head()

,title,year,runtime,genres,avg_rating,numvotes,director,actors
0,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Harnam Singh Rawail,Dilip Kumar
1,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Harnam Singh Rawail,Balraj Sahni
2,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Harnam Singh Rawail,Sanjeev Kumar
3,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43.0,Mani Kaul,Arun Khopkar
4,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43.0,Mani Kaul,Om Shivpuri


##### Merger Numbers and Box Office to source revenue for every movie

In [111]:
# bom_movie_df = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
# reviews_df = pd.read_csv('./zippedData/rt.reviews.tsv.gz',sep="\t", encoding = 'unicode_escape')
# tmdb_df = pd.read_csv('./zippedData/tmdb.movies.csv.gz')
# budget_df = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')


# Merge 'Box Office Mojo' with 'The Numbers' on a common column
merged_df = pd.merge(bom_movie_df, budget_df, how='inner', on='movie')
# Merge 'Box Office Mojo' with 'The Numbers' on 'movie_title' column
merged_df = pd.merge(box_office_mojo_df, the_numbers_df, how='inner', on='movie_title')

KeyError: 'movie'

In [ ]:
merged_df.head()